# Datalab - BigQuery - quick start

## Running queries

#### To start working with BigQuery in Datalab, just import `google.datalab.bigquery` library and run queries:

In [ ]:
import google.datalab.bigquery as bq
query = """SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips_agg`"""

df = bq.Query(query).execute().result().to_dataframe()
df.head()

#### Or use `%%bq` magic blocks. It helps to control costs with running `dryrun` to clculate the queried amount of data and verify if query results are cached already. 
#### Once query results are cached, running the same query doesn't cost the money.

In [ ]:
%%bq dryrun
SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips_agg`

In [ ]:
%%bq tables describe --name "siudzinskim-composer-demo-0.demo.taxi_trips_agg"

In [ ]:
%%bq query
SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips_agg`

## Use the partitioned tables and selective queries

#### Tables partitioning is extremly important in order to reduce the cost of queries. E.g. our source dataset `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018` is not partitioned and quite wide:

In [ ]:
%%bq tables describe --name "bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018"

#### Selecting all columns may cost quite a lot:

In [ ]:
%%bq dryrun
SELECT * FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018`

#### By selecting only required fields it is possible to reduce the cost of queries:

In [ ]:
%%bq dryrun 
SELECT 
    CAST(pickup_datetime AS DATE) AS pickup_datetime, 
    tip_amount, 
    total_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018`

#### If table is not partitioned, the whole dataset must be read to get results, even if you select data only from a single partition:

In [ ]:
%%bq dryrun 
SELECT 
    CAST(pickup_datetime AS DATE) AS pickup_datetime, 
    tip_amount, 
    total_amount
FROM `bigquery-public-data.new_york_taxi_trips.tlc_green_trips_2018`
WHERE DATE(pickup_datetime) >= DATE(2018,12,1)

#### That is why it's worth to create partitioned tables. E.g. `taxi_trips` table was created and partitioned by date:

In [ ]:
%%bq tables describe --name "siudzinskim-composer-demo-0.demo.taxi_trips"

#### Running query for all table will read all the partitions:

In [ ]:
%%bq dryrun
SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips`

#### But if only the most recent data is required, there is no need to read all the data. In following case it helps to save over 90% of money by selecting what we need:

In [ ]:
%%bq dryrun
SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips` WHERE pickup_datetime >= DATE(2018,12,1)

In [ ]:
%%bq query
SELECT * FROM `siudzinskim-composer-demo-0.demo.taxi_trips` WHERE pickup_datetime >= DATE(2018,12,1)

## Create and query simple ML models with SQL only

#### BigQuery allows to define simple Machine Learning models on SQL tables using only the SQL language:

In [ ]:
%%bq query 
DROP MODEL `siudzinskim-composer-demo-0.demo.total_model`

In [ ]:
%%bq dryrun
CREATE MODEL `demo.total_model`
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['avg_total']) AS
SELECT
  trips.trip_date,
  trips.avg_tips,
  trips.avg_total,
  weather.averageif_air_pressure,
  weather.averageif_temperature,
  weather.averageif_wind_angle,
  weather.averageif_wind_speed
FROM
  `siudzinskim-composer-demo-0.demo.taxi_trips_agg` trips
JOIN
  `siudzinskim-composer-demo-0.demo.nyc_avg_temperature` weather
ON
  trips.trip_date = DATE(weather.date)
  AND RAND() < 0.75


In [ ]:
%%bq query
CREATE MODEL `demo.total_model`
OPTIONS
  (model_type='linear_reg',
    input_label_cols=['avg_total']) AS
SELECT
  trips.trip_date,
  trips.avg_tips,
  trips.avg_total,
  weather.averageif_air_pressure,
  weather.averageif_temperature,
  weather.averageif_wind_angle,
  weather.averageif_wind_speed
FROM
  `siudzinskim-composer-demo-0.demo.taxi_trips_agg` trips
JOIN
  `siudzinskim-composer-demo-0.demo.nyc_avg_temperature` weather
ON
  trips.trip_date = DATE(weather.date)
  AND RAND() < 0.75


In [ ]:
%%bq query
SELECT
  trip_date ,
  avg_total,
  predicted_avg_total,
  ABS(avg_total - predicted_avg_total) AS difference
FROM
  ML.PREDICT(MODEL `demo.total_model`,
(
SELECT
  trips.trip_date,
  trips.avg_tips,
  trips.avg_total,
  weather.averageif_air_pressure,
  weather.averageif_temperature,
  weather.averageif_wind_angle,
  weather.averageif_wind_speed
FROM
  `siudzinskim-composer-demo-0.demo.taxi_trips_agg` trips
JOIN
  `siudzinskim-composer-demo-0.demo.nyc_avg_temperature` weather
ON
  trips.trip_date = DATE(weather.date)
))


### ENJOY YOU JOURNEY WITH BIGQUERY